## This notebook consists of generating both static and temporal segment tolerances using only one of the five thermal maps recieved from L3 Harris Technologies.

### Importing necessary  python libraries, and PASTIS pre-built functions

In [ ]:
import os
os.chdir("/Users/asahoo/repos/PASTIS")
import time
from shutil import copy
from astropy.io import fits
import astropy.units as u
import hcipy
import numpy as np
import pastis.util as util    
from pastis.config import CONFIG_PASTIS 
from pastis.e2e_simulators.luvoir_imaging import LuvoirA_APLC 
from pastis.e2e_simulators.generic_segmented_telescopes import SegmentedAPLC
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import griddata
import exoscene.image
import exoscene.star
import exoscene.planet
from exoscene.planet import Planet
from astropy.io import fits as pf

### Set some initial parameter or call them from config file

In [ ]:
coronagraph_design = 'small'
nb_seg = CONFIG_PASTIS.getint('LUVOIR', 'nb_subapertures')
nm_aber = CONFIG_PASTIS.getfloat('LUVOIR', 'calibration_aberration') * 1e-9
sampling = CONFIG_PASTIS.getfloat('LUVOIR', 'sampling')

### Load thermal mode files from L3 Harris Technologies 

In [ ]:
filepath = "/Users/asahoo/Documents/ultra/Sensitivities2.xlsx"
pad_orientation = np.pi/2*np.ones(nb_seg)

###  Define and create directory

In [ ]:
data_dir = "/Users/asahoo/Desktop/data_repos/harris_data"
repo_dir = "/Users/asahoo/repos/PASTIS"
overall_dir = util.create_data_path(data_dir, telescope='luvoir_'+coronagraph_design)
resDir = os.path.join(overall_dir, 'matrix_numerical')

# Create necessary directories if they don't exist yet
os.makedirs(resDir, exist_ok=True)

### Instantiate LUVOIR-A

In [ ]:
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
luvoir = LuvoirA_APLC(optics_input, coronagraph_design, sampling)

### Create harris deformable mirror 

In [ ]:
luvoir.create_segmented_harris_mirror(filepath,pad_orientation, thermal = True,mechanical=False,other=False)
luvoir.harris_sm #hcipy DM
n_harris = luvoir.harris_sm.num_actuators

### Flatten the DM

In [ ]:
luvoir.harris_sm.flatten()

After applying a random actuator setting to 600 actuators i.e. (luvoir.harris_sm.random(1000)), (luvoir.harris_sm.phase)seems to work fine but Harris mode mirror phase called inside luvoir.calc_psf is not working

### Calculate the unaberrated coronagraphic PSF

In [ ]:
unaberrated_coro_psf, ref= luvoir.calc_psf(ref=True, display_intermediate=True, norm_one_photon=True)

### Calculate peak value of reference psf and static coronaraphic floor $c_{0}$

In [ ]:
norm = np.max(ref)
dh_intensity = (unaberrated_coro_psf / norm) * luvoir.dh_mask
contrast_floor = np.mean(dh_intensity[np.where(luvoir.dh_mask != 0)])
print(f'norm: {norm}',f'constrast floor: {contrast_floor}')

### Poking each segment with a thermal aberration and generating the subsequent electric field¶

In [ ]:
nonaberrated_coro_psf, ref, efield = luvoir.calc_psf(ref=True, display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
Efield_ref = nonaberrated_coro_psf.electric_field

In [ ]:
print('Generating the E-fields for harris modes in science plane')
print(f'Calibration aberration used: {nm_aber} m')

start_time = time.time()
focus_fieldS = []
focus_fieldS_Re = []
focus_fieldS_Im = []

for i in range(0, 120):
    print(f'Working on "bulk" thermal mode, segment: {i+1}')
    
    # Apply calibration aberration to used mode
    harris_mode = np.zeros(n_harris)
    harris_mode[5*i] = (nm_aber)/2 
    luvoir.harris_sm.actuators  = harris_mode
   
    # Calculate coronagraphic E-field and add to lists
    aberrated_coro_psf, inter = luvoir.calc_psf(display_intermediate=False, return_intermediate='efield',norm_one_photon=True)
    focus_field1 = aberrated_coro_psf
    focus_fieldS.append(focus_field1)
    focus_fieldS_Re.append(focus_field1.real)
    focus_fieldS_Im.append(focus_field1.imag)

### Taking a pair of poked efields to compute PASTIS matrix

In [ ]:
mat_bulk = np.zeros([120, 120])
for i in range(0, 120):
    for j in range(0, 120):
        test = np.real((focus_fieldS[i].electric_field - Efield_ref) * np.conj(focus_fieldS[j].electric_field - Efield_ref))
        dh_test = (test / norm) * luvoir.dh_mask
        contrast = np.mean(dh_test[np.where(luvoir.dh_mask != 0)])
        mat_bulk[i, j] = contrast



In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(np.log10(np.abs(mat_bulk)))
plt.title(r"PASTIS matrix $M$", fontsize=20)
plt.xlabel("Mode Index",fontsize=20)
plt.ylabel("Mode Index",fontsize=20)
plt.tick_params(labelsize=15)
plt.tight_layout()
plt.colorbar()

### Codeblock to generate .gif file

In [ ]:
# for i in range(0, 120):
#     print(f'Working on "bulk" thermal mode, segment: {i+1}')
    
#     # Apply calibration aberration to used mode
#     harris_mode = np.zeros(n_harris)
#     harris_mode[i*5] = (nm_aber)/2 
#     luvoir.harris_sm.actuators  = harris_mode
    
#     plt.figure()
#     plt.text(9.6, 7.6, 'm', fontsize = 10)
#     hcipy.imshow_field(luvoir.harris_sm.surface, mask=luvoir.aperture, cmap='RdBu')
#     cbar = plt.colorbar()
#     plt.tight_layout()
#     #plt.savefig('/Users/asahoo/Documents/ultra/temp_plots/bulk_2_2_22/poke_%d.png'% i, dpi=165)
#     plt.savefig('/Users/asahoo/Documents/ultra/temp_plots/bulk_2_2_22/poke_{0:03}.png'.format(i), dpi=165)
    

In [ ]:
# from PIL import Image
# import glob
# os.chdir("/Users/asahoo/Documents/ultra/temp_plots/bulk_2_2_22/")

# imgs = glob.glob("/Users/asahoo/Documents/ultra/temp_plots/bulk_2_2_22/*.png")
# imgs.sort()

# for i in imgs:
#     new_frame = Image.open(i)
#     frames.append(new_frame)
 
# # Save into a GIF file that loops forever
# frames[0].save('bulk.gif', format='GIF',
#                append_images=frames[1:],
#                save_all=True,
#                duration=100, loop=0)